# Top spending users
Data Analysis Python Pandas Data Manipulation External Dataset

Suppose you have the following
[dataset](https://docs.google.com/spreadsheets/d/1DrvkAWnO1psWkFN1YVt891sHe4yHl4ljNPUVlCsI95M/edit#gid=2039795889) 
which contains (1st tab) a list of items purchased by a given user,
(2nd tab) a mapping which maps the item_id to the item name and price,
(3rd tab) a matrix that formats data from sheet 1 into a matrix with users
in rows and the number of each item_id purchased in columns.

Given these 3 datasets, can you rank the users in descending order based on spend?

In [1]:
import pandas as pd

filename = 'q143_data.csv'
df = pd.read_csv(filename)
df.head()

,user_id,id
0,222087,"27,26"
1,1343649,"6,47,17"
2,404134,"18,12,23,22,27,43,38,20,35,1"
3,1110200,"9,23,2,20,26,47,37"
4,224107,"31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4..."


In [2]:
# 2 steps: 1) break down id1,id2 into 2 rows, then 2) pivot df

# step 1: https://stackoverflow.com/a/28182629
purchases_df = (
    pd.DataFrame(
        df['id'].str.split(',').tolist(), 
        index=df['user_id']
    )
    .stack()
    .reset_index()
    .rename(columns={0:'item_id'})
    [['user_id','item_id']]
)
purchases_df['item_id'] = purchases_df['item_id'].astype(int)
purchases_df.head()

,user_id,item_id
0,222087,27
1,222087,26
2,1343649,6
3,1343649,47
4,1343649,17


In [3]:
prices_df = pd.read_csv('q144_data.csv')
prices_df['item_id'] = prices_df['Item_id'].astype('int')
prices_df = prices_df[['item_id', 'price']]
prices_df.head()

,item_id,price
0,1,2
1,2,1
2,3,2
3,4,4
4,5,2


In [4]:
qty_df = (
    purchases_df
    .groupby(['user_id','item_id'])
    .size()
    .reset_index(name='qty')
    .sort_values(by='qty', ascending=False) # just to show that some users bought the same item multiple times
    .reset_index(drop=True)
)
qty_df.head()

,user_id,item_id,qty
0,269335,2,5
1,599172,39,5
2,1198106,45,5
3,917199,18,5
4,920002,23,5


In [5]:
joined_df = pd.merge(qty_df, prices_df, how='inner', left_on='item_id', right_on='item_id')
joined_df.head()

,user_id,item_id,qty,price
0,269335,2,5,1
1,31625,2,5,1
2,540483,2,4,1
3,1141602,2,4,1
4,909111,2,4,1


In [6]:
joined_df['spend'] = joined_df['qty'] * joined_df['price']
joined_df.groupby('user_id').agg({'spend':'sum'}).sort_values(by='spend', ascending=False).head(5)

,spend
user_id,
917199,169
367872,163
377284,161
269335,160
397623,156
